In [ ]:
import { parseHTML } from "npm:linkedom";
import { Database, Statement } from "jsr:@db/sqlite@0.11";
import * as sqlitePath from "npm:sqlite-path";
import * as sqliteUrl from "npm:sqlite-url";
import * as sqliteRegex from "npm:sqlite-regex";

In [ ]:
const months = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]

const db = new Database(":memory:")
db.enableLoadExtension = true;
db.loadExtension(sqlitePath.getLoadablePath());
db.loadExtension(sqliteUrl.getLoadablePath());
db.loadExtension(sqliteRegex.getLoadablePath());
db.enableLoadExtension = false;

db.exec(`
  CREATE TABLE articles(
    slug_id TEXT,
    slug TEXT,
    headline TEXT,
    url TEXT,
    year integer,
    month integer,
    category1 TEXT,
    category2 TEXT
  )
`)

const stmt = db.prepare(`
  insert into articles
  select
    regex_capture(
      '(?P<slug>.+)-(?P<id>rcna\\d+)',
      path_at(url_path(:url), -1),
      'id'
    ) as id,
     regex_capture(
      '(?P<slug>.+)-(?P<id>rcna\\d+)',
      path_at(url_path(:url), -1),
      'slug'
    ) as slug,
    :headline as headline,
    :url as url,
    :year as year,
    :month as month,
    path_at(url_path(:url), 0) as category1,
    iif(
      path_length(url_path(:url)) > 2,
      path_at(url_path(:url), 1),
      null
    ) as category2
`);

const insertArticles = db.transaction((year, month, articles) => {
  for(const article of articles) {
    stmt.run({...article, year, month})
  }
})

async function insertMonth(year:number, month: text) {
  const monthPage = await fetch(`https://www.nbcnews.com/archive/articles/${year}/${month}`).then(r=>r.text())
  const {document:monthPageDoc} = parseHTML(monthPage);
  const monthEntries = monthPageDoc
    .querySelectorAll('.MonthPage a')
    .map(a => ({headline: a.innerText, url: a.getAttribute('href')}));
    insertArticles(year, months.findIndex(m => m === month)+1, monthEntries)
}


In [ ]:

for(let year = 2014; year <= 2023; year++) {
  for(const month of months) {
    console.log(year, month);
    await insertMonth(year, month);
  }
}


In [ ]:
db.exec("vacuum into 'articles.db'")

In [ ]:
db.sql`select * from articles order by random() limit 10`